1. Install the Vertex AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [ ]:
!pip install --upgrade google-genai

2. Use the following code in your application to request a model response

Running through profiles:

In [ ]:
from google import genai
from google.genai import types
import json
import re  # Import the regular expression module
import datetime # For timestamped logging

def generate():
    print(f"[{datetime.datetime.now()}] --- Starting Profile-Based Risk Assessment --- ")
    # This function loads company and person profiles from 'profiles.json',
    # generates search queries using predefined templates, interacts with a GenAI model
    # to gather information and assess risk, and saves the results to 'raw_research.txt'
    # and 'results.txt'.

    # Create or empty the output files at the start
    raw_research_file = "./content/raw_research.txt"
    results_file = "./content/results.txt"
    profiles_file = "./content/profiles.json"

    print(f"[{datetime.datetime.now()}] Initializing output file: {raw_research_file}")
    with open(raw_research_file, "w") as f: # Output file for detailed search data
        f.write("")  # Write nothing to empty the file

    print(f"[{datetime.datetime.now()}] Initializing output file: {results_file}")
    with open(results_file, "w") as f: # Output file for risk assessment summaries
        f.write("")  # Write nothing to empty the file

    # Initialize the GenAI Client
    print(f"[{datetime.datetime.now()}] Initializing GenAI Client...")
    client = genai.Client(
        vertexai=True, # Specify to use Vertex AI
        project="agenthack-05", # GCP Project ID
        location="us-central1", # GCP Location
    )
    print(f"[{datetime.datetime.now()}] GenAI Client initialized.")

    # Configure the GenAI model and tools
    model = "gemini-2.0-flash-001" # Specify the model to use
    tools = [types.Tool(google_search=types.GoogleSearch())] # Enable Google Search as a tool
    generate_content_config = types.GenerateContentConfig(
        temperature=0.5, # Controls randomness, lower is more deterministic
        top_p=0.95, # Nucleus sampling parameter
        max_output_tokens=8192, # Maximum number of tokens in the response
        response_modalities=["TEXT"], # Expect text responses
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ], # Disable safety filters for this specific use case (use with caution)
        tools=tools, # Provide the configured tools to the model
    )
    print(f"[{datetime.datetime.now()}] Model and tools configured.")

    # Load profiles from profiles.json
    print(f"[{datetime.datetime.now()}] Loading profiles from {profiles_file}...")
    try:
        with open(profiles_file, "r") as f:
            profiles_data = json.load(f)
            profiles = profiles_data.get("profiles", {})  # Use .get() for safety, expecting a 'profiles' key
        print(f"[{datetime.datetime.now()}] Successfully loaded {len(profiles)} company profiles.")
    except FileNotFoundError:
        print(f"[{datetime.datetime.now()}] Error: {profiles_file} not found in the current directory.")
        return
    except json.JSONDecodeError as e:
        print(f"[{datetime.datetime.now()}] Error: {profiles_file} is not valid JSON. Details: {e}")
        return
    except Exception as e:
        print(f"[{datetime.datetime.now()}] An unexpected error occurred while loading {profiles_file}: {e}")
        return

    if not isinstance(profiles, dict):
        print(f"[{datetime.datetime.now()}] Error: 'profiles' key in {profiles_file} must contain a dictionary.")
        return

    # Predefined prompt templates for generating search queries
    prompt_templates = [
      "<person_name> (<company_name>) bankruptcy",
      "<person_name> (<company_name>) fraud",
      "<person_name> (<company_name>) jail",
      "<person_name> (<company_name>) lawsuit",
      "<person_name> (<company_name>) debt",
      "<person_name> (<company_name>) court records",
      "<person_name> (<company_name>) criminal history",
      "<person_name> (<company_name>) identity theft",
      "<person_name> (<company_name>) lien records",
      "<person_name> (<company_name>) crime",
      "<person_name> (<company_name>) trial",
      "<person_name> (<company_name>) terrorist",
      "<person_name> (<company_name>) trafficking",
      "<person_name> (<company_name>) scandal",
      "<person_name> (<company_name>) theft",
      "<person_name> (<company_name>) illegal",
      "<person_name> (<company_name>) guilty",
      "<person_name> (<company_name>) cartel",
      "<person_name> (<company_name>) offshore",
      "<person_name> (<company_name>) sanction",
      "<person_name> (<company_name>) corrupt",
      "<person_name> (<company_name>) sued",
      "<person_name> (<company_name>) arrest",
      "<person_name> (<company_name>) convicted",
      "<person_name> (<company_name>) extremist",
      "<person_name> (<company_name>) scam",
      "<person_name> (<company_name>) smuggle",
      "<person_name> (<company_name>) bribe",
      "<person_name> (<company_name>) fined",
      "<person_name> (<company_name>) mafia",
      "<person_name> (<company_name>) laundering",
      "<person_name> (<company_name>) manipulation"
    ]

    # Iterate through each company and its associated persons from the profiles
    print(f"[{datetime.datetime.now()}] Starting profile processing loop...")
    for company_name, persons in profiles.items():
        print(f"[{datetime.datetime.now()}] Processing company: {company_name}")
        if not isinstance(persons, list):
            print(f"[{datetime.datetime.now()}] Warning: Skipping company '{company_name}' because its value is not a list of persons.")
            continue

        for person_name_original in persons:
            if not isinstance(person_name_original, str):
                print(f"[{datetime.datetime.now()}] Warning: Skipping non-string person name ('{person_name_original}') in company '{company_name}'.")
                continue

            # Handle placeholder for company name within person_name list
            if person_name_original.startswith("<") and person_name_original.endswith(">"):
                placeholder = person_name_original[1:-1]
                if placeholder == "company_name":
                    person_name_resolved = company_name # Use the actual company name
                    print(f"[{datetime.datetime.now()}] Resolved placeholder '<company_name>' to '{company_name}' for company '{company_name}'.")
                else:
                    print(f"[{datetime.datetime.now()}] Warning: Unknown placeholder '{person_name_original}' found for company '{company_name}'. Skipping person.")
                    continue
            else:
                person_name_resolved = person_name_original

            print(f"[{datetime.datetime.now()}] Processing person: {person_name_resolved} (Original: {person_name_original}) for company: {company_name}")

            person_profile_data = f"""
            Risk profile source data for: {person_name_resolved} (Company: {company_name})
            """

            # Generate and process each prompt for the current person
            for template_idx, template in enumerate(prompt_templates):
                # Create the specific prompt by replacing placeholders
                prompt = template.replace("<person_name>", person_name_resolved).replace(
                    "<company_name>", company_name
                )
                contents = prompt  # The prompt itself is the content for the GenAI model
                print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] Generating content for prompt {template_idx + 1}/{len(prompt_templates)}: '{prompt}'")

                # The 'prompt' variable is already defined before this block.
                # 'contents' is also defined as 'prompt'.
                print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] Preparing to call GenAI for prompt: '{prompt}'")
                person_profile_data += "=========================\n\n"
                person_profile_data += f"Input prompt: {prompt}\n"

                # TODO: Call the GenAI model for information gathering and extract its text response.
                # 1. Use 'client.models.generate_content()' to make the API call.
                # 2. For the call, use the 'model' variable (currently "gemini-2.0-flash-001") and the 'generate_content_config' variable.
                #    The 'contents' variable (which is the 'prompt') should be passed as the content.
                # 3. Extract the primary text content from the model's response.
                #    (Hint: The text is usually in `response.candidates[0].content.parts[0].text`)
                # 4. Store this extracted text in a variable, e.g., 'model_response_text'.
                # 5. Append 'model_response_text' to 'person_profile_data' under a "Generated response to the prompt:" heading.
                # 6. Implement try-except error handling around the API call and response processing.
                #    If an error occurs, append an error message to 'person_profile_data'.

                # Placeholder for the model's response text:
                # model_response_text = "[TODO: Implement GenAI call and place its text response here]"
                # Example of appending (actual implementation will be within the try-except block):
                try:
                  response = client.models.generate_content(model=model, contents=contents, config=generate_content_config)
                  model_response_text = response.candidates[0].content.parts[0].text
                  print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] Received response for prompt: '{prompt}'")
                except Exception as e:
                  model_response_text = f"Error processing prompt '{prompt}': {e}"
                  print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] {model_response_text}")

                person_profile_data += "\n\nGenerated response to the prompt:\n"
                person_profile_data += model_response_text
                person_profile_data += "\n------End of response for this prompt------\n\n"

            # Append the collected data for this person to 'raw_research.txt'
            print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] Appending research data to {raw_research_file}...")
            with open(raw_research_file, "a") as f:
                f.write(person_profile_data)
            print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] Research data appended.")

            financial_risk_results = """ """ # Initialize string for risk assessment results
            try:
                print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] Preparing for risk assessment call...")
                # --- Risk Assessment Call using the gathered context ---
                risk_assessment_prompt = f"""Based on the following context only, assess the risk profile of {person_name_resolved} of {company_name}.

                You must respond in the following format:
                ** High risk factors:**
                - Brief description of why this person has a high risk
                ** Neutral discoveries: **
                - Brief description of things that are nice to know but don't pose a high risk
                ** Positive discoveries: **
                - Brief description of things that are positive and make this a low to neutral risk profile
                ** Conclusion **
                - Brief paragraph on the risk profile.
                If there is no data available in the context provided, you must simply respond: "No data points found based on the keywords searched - this could indicate a low to neutral risk profile"
======================
CONTEXT:
======================
{person_profile_data}"""

                # Configuration for the risk assessment model call (can be same as above or different)
                model_risk = "gemini-2.0-flash-001"
                generate_content_config_risk = types.GenerateContentConfig(
                  temperature = 0.5,
                  top_p = 0.95,
                  max_output_tokens = 8192,
                  response_modalities = ["TEXT"],
                  safety_settings = [types.SafetySetting(
                    category="HARM_CATEGORY_HATE_SPEECH",
                    threshold="OFF"
                  ),types.SafetySetting(
                    category="HARM_CATEGORY_DANGEROUS_CONTENT",
                    threshold="OFF"
                  ),types.SafetySetting(
                    category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
                    threshold="OFF"
                  ),types.SafetySetting(
                    category="HARM_CATEGORY_HARASSMENT",
                    threshold="OFF"
                  )],
                )

                print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] Sending risk assessment request to GenAI model...")

                # TODO: Call the GenAI model for risk assessment and extract its text response.
                # 1. Use 'client.models.generate_content()' to make the API call.
                # 2. For the call, use the 'model_risk' variable (currently "gemini-2.0-flash-001"), the 'risk_assessment_prompt' variable,
                #    and the 'generate_content_config_risk' variable.
                # 3. Extract the primary text content from the model's response.
                #    (Hint: The text is usually in `risk_response.candidates[0].content.parts[0].text`)
                # 4. Store this extracted text in a variable, e.g., 'assessment_text'.
                # 5. This 'assessment_text' will then be appended to 'financial_risk_results'.
                # (The existing try-except block will handle errors)

                # Placeholder for the risk assessment text:
                # assessment_text = "[TODO: Implement GenAI call for risk assessment and place its text response here]"
                # Example (actual implementation will replace the line above):
                risk_response = client.models.generate_content(
                    model=model_risk,
                    contents=risk_assessment_prompt,
                    config=generate_content_config_risk,
                )
                assessment_text = risk_response.candidates[0].content.parts[0].text
                print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] Received risk assessment response.")

                financial_risk_results += "====================================\n====================================\n"
                financial_risk_results += f"\n--- Financial Risk Assessment for {person_name_resolved} of {company_name} ---\n"
                financial_risk_results += assessment_text + "\n" # Appending the (to be implemented) assessment_text
                if assessment_text != "[TODO: Implement GenAI call for risk assessment and place its text response here]": # Basic check if implemented
                     print(f"\n[{datetime.datetime.now()}] --- Financial Risk Assessment for {person_name_resolved} of {company_name} ---") # Original print
                     print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] Extracted risk assessment details.")
                else:
                     print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] No risk assessment generated (TODO pending).")

            except Exception as e:
                error_message = f"An error occurred during risk assessment: {e}"
                print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] {error_message}")
                financial_risk_results += f"\n--- Financial Risk Assessment for {person_name_resolved} of {company_name} ---\n"
                financial_risk_results += error_message + "\n"

            # Append the financial risk assessment to 'results.txt'
            print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] Appending risk assessment to {results_file}...")
            with open(results_file, "a") as f:
                f.write(financial_risk_results)
            print(f"[{datetime.datetime.now()}] [{company_name} - {person_name_resolved}] Risk assessment appended.")
    print(f"[{datetime.datetime.now()}] --- Finished processing all profiles. ---")


In [9]:
# Call the main generate function to run the profile-based risk assessment
for i in range(0, 1): # Runs the assessment once
  generate()

[2025-06-05 15:27:52.740091] --- Starting Profile-Based Risk Assessment --- 
[2025-06-05 15:27:52.740157] Initializing output file: ./content/raw_research.txt
[2025-06-05 15:27:52.740612] Initializing output file: ./content/results.txt
[2025-06-05 15:27:52.740876] Initializing GenAI Client...
[2025-06-05 15:27:52.796250] GenAI Client initialized.
[2025-06-05 15:27:52.796378] Model and tools configured.
[2025-06-05 15:27:52.796390] Loading profiles from ./content/profiles.json...
[2025-06-05 15:27:52.796474] Successfully loaded 7 company profiles.
[2025-06-05 15:27:52.796481] Starting profile processing loop...
[2025-06-05 15:27:52.796485] Processing company: Deepmind
[2025-06-05 15:27:52.796489] Processing person: Demis Hassabis (Original: Demis Hassabis) for company: Deepmind
[2025-06-05 15:27:52.796496] [Deepmind - Demis Hassabis] Generating content for prompt 1/32: 'Demis Hassabis (Deepmind) bankruptcy'
[2025-06-05 15:27:52.796501] [Deepmind - Demis Hassabis] Preparing to call GenAI

/home/egorman/git/agenthack_adk_boilerplate/.venv/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


[2025-06-05 15:27:55.338409] [Deepmind - Demis Hassabis] Error processing prompt 'Demis Hassabis (Deepmind) bankruptcy': 403 PERMISSION_DENIED. {'error': {'code': 403, 'message': "Permission 'aiplatform.endpoints.predict' denied on resource '//aiplatform.googleapis.com/projects/agenthack-31/locations/us-central1/publishers/google/models/gemini-2.0-flash-001' (or it may not exist).", 'status': 'PERMISSION_DENIED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'IAM_PERMISSION_DENIED', 'domain': 'aiplatform.googleapis.com', 'metadata': {'resource': 'projects/agenthack-31/locations/us-central1/publishers/google/models/gemini-2.0-flash-001', 'permission': 'aiplatform.endpoints.predict'}}]}}
[2025-06-05 15:27:55.338653] [Deepmind - Demis Hassabis] Generating content for prompt 2/32: 'Demis Hassabis (Deepmind) fraud'
[2025-06-05 15:27:55.338676] [Deepmind - Demis Hassabis] Preparing to call GenAI for prompt: 'Demis Hassabis (Deepmind) fraud'
[2025-06-05 15:27:55.